In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

In [2]:
print(f"Is there a gpu available? {'Yes' if torch.cuda.is_available() else 'No'}")

print(f"Device Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

print(f"Memory allocated in bytes: {torch.cuda.memory_allocated()}")

Is there a gpu available? Yes
Device Name: NVIDIA GeForce RTX 4060 Laptop GPU
Memory allocated in bytes: 0


In [3]:
a = torch.tensor([1,23]).cuda()
a.device

device(type='cuda', index=0)

In [4]:
class Model(nn.Module):

    def __init__(self, in_features = 4, h1=8, h2=9, out_features = 3):
        super().__init__()

        self.fc1 = nn.Linear(in_features,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.fc3 = nn.Linear(h2,out_features)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)

        return X

In [5]:
torch.manual_seed(32)
model = Model()
gpumodel = model.cuda()

next(gpumodel.parameters()).is_cuda

True

In [6]:
next(model.parameters()).is_cuda

True

In [7]:
data = load_iris()
X = pd.DataFrame(data['data'], columns=data['feature_names'])
y = pd.Series(data['target'])
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X.values,y.values, test_size=0.2, random_state=33)
print(f"Shape")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")

Shape
X_train: (120, 4)
X_test: (30, 4)


In [9]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.FloatTensor(y_train).cuda().long()
y_test = torch.FloatTensor(y_test).cuda().long()

In [10]:
y_train[:4]

tensor([1, 1, 2, 2], device='cuda:0')

In [11]:
def loader_info(loader: DataLoader) -> None:
    print(f"Number of Batches: {len(loader)}")
    print(f"Size of each batch: {loader.batch_size}\n")

trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

loader_info(trainloader)
loader_info(testloader)

Number of Batches: 2
Size of each batch: 60

Number of Batches: 1
Size of each batch: 60



In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = 0.01)

In [13]:
import time
start = time.time()

epochs = 100
losses = []

for i in range(epochs):

    y_pred = gpumodel(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss.item())

    if i%10 == 0:
        print(f"Epoch {i} | loss: {loss.item()}")

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

total = time.time() - start
print(f"Duration: {total/60} mins")

Epoch 0 | loss: 1.1507450342178345
Epoch 10 | loss: 0.9371446371078491
Epoch 20 | loss: 0.779624342918396
Epoch 30 | loss: 0.6078532338142395
Epoch 40 | loss: 0.39894530177116394
Epoch 50 | loss: 0.2524919807910919
Epoch 60 | loss: 0.14927688241004944
Epoch 70 | loss: 0.10029558837413788
Epoch 80 | loss: 0.08100691437721252
Epoch 90 | loss: 0.07216038554906845
Duration: 0.0030820250511169435 mins


In [15]:
correct = 0
with torch.no_grad():
    for i, data in enumerate(X_test):
        y_val = gpumodel.forward(data)
        print(f"{i+1:2}. {str(y_val):38} {y_test[i]}")
        if y_val.argmax().item() == y_test[i]:
            correct+=1

print(f"\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct")


 1. tensor([-2.1234,  4.8067, -0.8803], device='cuda:0') 1
 2. tensor([-1.7920,  5.3100, -1.5693], device='cuda:0') 1
 3. tensor([  6.3723,   0.8741, -10.0971], device='cuda:0') 0
 4. tensor([-3.9129,  4.5951,  1.1509], device='cuda:0') 1
 5. tensor([-7.4882,  3.1953,  5.7839], device='cuda:0') 2
 6. tensor([-10.5202,   1.6381,   9.6291], device='cuda:0') 2
 7. tensor([  6.3364,   1.0237, -10.1951], device='cuda:0') 0
 8. tensor([  7.0690,   0.7370, -10.9620], device='cuda:0') 0
 9. tensor([-7.2218,  3.3422,  5.3528], device='cuda:0') 2
10. tensor([-9.4170,  2.5675,  8.1028], device='cuda:0') 2
11. tensor([-9.9029,  2.3388,  8.7141], device='cuda:0') 2
12. tensor([ 6.2942,  0.6938, -9.8046], device='cuda:0') 0
13. tensor([-9.3335,  2.1817,  8.1917], device='cuda:0') 2
14. tensor([-3.7832,  4.5046,  1.0603], device='cuda:0') 1
15. tensor([-7.8793,  3.0060,  6.2225], device='cuda:0') 2
16. tensor([-1.8810,  5.1571, -1.3572], device='cuda:0') 1
17. tensor([-5.7107,  3.5003,  3.6612], devi